In [10]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import json
from math import isnan
import pandas as pd
from plotly_resampler import FigureResampler
import os
from datetime import datetime
import plotly.express as px
from tqdm import tqdm
import pickle
from copy import deepcopy

def read_hci_profile(raw_data, key = 'hci'):
    points = {
        'time_key': [],
        'value': [],
        'md': [],
    }
    for time_key, mds, hcis in zip(raw_data['time_key'], raw_data['md'], raw_data[key]):
        for md, hci in zip(mds, hcis):
            points['md'].append(md)
            points['time_key'].append(time_key)
            points['value'].append(hci)

    points = pd.DataFrame(points)

    return points   

def read_ecd_profile(raw_data):
    points = {
        'time_key': [],
        'value': [],
        'md': [],
    }
    for item in tqdm(raw_data):
        time_key = item['Key']
        item_output = item['Value']
        if 'RTHydro.PressureProfile' in list(item_output.keys()):
            pressure_profile = json.loads(item_output['RTHydro.PressureProfile'])['ProfilePoints']

            for profile_point in pressure_profile:
                if (isnan(float(profile_point['Ecd']['P50']))):
                    continue
                points['md'].append(float(profile_point['MD']))
                points['time_key'].append(time_key)
                points['value'].append(float(profile_point['Ecd']['P50']))

    # assert len(points['md']) == len(points['time_key'])
    # assert len(points['md']) == len(points['value'])

    points = pd.DataFrame(points)

    return points   

def read_profile(raw_data):
    points = {
        'md': [],
        'time_key': [],
        'ecd': []
    } 

    hci_points = {
        'md': [],
        'time_key': [],
        'hci': [],
        'cc': [],
        'cbh': []
    }
    for item in tqdm(raw_data):
        time_key = item['Key']
        item_output = item['Value']
        if 'RTHydro.PressureProfile' in list(item_output.keys()):
            pressure_profile = json.loads(item_output['RTHydro.PressureProfile'])['ProfilePoints']

            for profile_point in pressure_profile:
                if (isnan(float(profile_point['Ecd']['P50']))):
                    continue
                points['md'].append(float(profile_point['MD']))
                points['time_key'].append(time_key)
                points['ecd'].append(float(profile_point['Ecd']['P50']))
        if 'RTHydro.HoleCleaningProfile' in list(item_output.keys()):
            hci_profile = json.loads(item_output['RTHydro.HoleCleaningProfile'])
            for md, hci, cbh, cc in zip(hci_profile['MD'], hci_profile['HCI'], hci_profile['CuttingBedHeightP50'], hci_profile['CuttingConcentrationP50']):

                hci_points['md'].append(float(md))
                hci_points['time_key'].append(time_key)
                hci_points['hci'].append(float(hci))
                hci_points['cbh'].append(float(cbh))
                hci_points['cc'].append(float(cc))

    points = pd.DataFrame(points)

    return points, hci_points

def read_outputs(raw_data):
    points = {
        'time_key': [],
        'drill_string_pressure_loss': [],
        'tubular_pressure_loss': [],
        'bit_pressure_loss': [],
        'tools_pressure_loss': [],
        'rss_pressure_loss': [],
        'motor_pressure_loss': [],
        'sspp': [],
        'ecd_at_bit': []
    }
    for item in raw_data:
        time_key = item['Key']
        item_output = item['Value']
        # if item_output.['HydraulicsTransientSimulation']
        # ['RTHydro.SPPSIM', 'RTHydro.ATBITECDSIM', 'RTHydro.HCI', 'RTHydro.MaxHCIDepth', 'RTHydro.CUTBED', 'RTHydro.CUTCON', 'RTHydro.PressureProfile', 'RTHydro.HoleCleaningProfile']
        # ['AnnulusPressureDrop', 'TooljointAnnulusPressureDrop', 'CasingProtectorPressureDrop', 'CuttingsPressureDrop', 'DrillstringPressureDrop', 'TooljointDrillstringPressureDrop', 'TubularsPressureDrop', 
        # 'StandpipePressure', 'BitPressureDrop', 'ToolsPressureDrop', 'RssPressureDrop', 'FlowRestrictorPressureDrop', 'MotorPressureDrop', 'TurbinePressureDrop', 'SurfaceEquipmentPressureDrop', 'HydraulicImbalance']
        if 'RTHydro.PressureProfile' in list(item_output.keys()) and 'RTHydro.SPPSIM' in list(item_output.keys()):
            pressure_drop_all = json.loads(item_output['RTHydro.PressureProfile'])['PressureDrop']
            if pressure_drop_all is None:
                continue
            drill_string_pressure_loss = pressure_drop_all['DrillstringPressureDrop']
            tubular_pressure_loss = pressure_drop_all['TubularsPressureDrop']
            points['time_key'].append(time_key)
            points['drill_string_pressure_loss'].append(drill_string_pressure_loss)
            points['tubular_pressure_loss'].append(tubular_pressure_loss)
            points['bit_pressure_loss'].append(pressure_drop_all['BitPressureDrop'])
            points['tools_pressure_loss'].append(pressure_drop_all['ToolsPressureDrop'])
            points['rss_pressure_loss'].append(pressure_drop_all['RssPressureDrop'])
            points['motor_pressure_loss'].append(pressure_drop_all['MotorPressureDrop'])
            points['sspp'].append(item_output['RTHydro.SPPSIM'])
            points['ecd_at_bit'].append(item_output['RTHydro.ATBITECDSIM'])

    points = pd.DataFrame(points)

    return points   

def every_nth(nums: dict, nth):
    # Use list slicing to return elements starting from the (nth-1) index, with a step of 'nth'.
    res = {}
    for k, v in nums.items():
        res[k] = nums[k][nth - 1::nth]
    return res

def filter_by_time_key(df, min_time, max_time):
    return df[(df['time_key'] > min_time) & (df['time_key'] < max_time)]

In [11]:
well_name = '3s-617_Run3'
well_data_path = r"C:\NotOneDrive\Data\merged_input_output_channels\3s-617_Run3_bha_TransientHydraulics_profile.json"



if '617_Run3' in well_name:
    packoff_time_key = '2023-11-18T12:00:00Z'
    time_key_display_max_limit = '2023-11-18T13:00:00Z'
    time_key_display_min_limit = '2023-11-14T12:00:00Z'
elif '624_Run3' in well_name:
    packoff_time_key = '2024-01-02T12:00:00Z'
    time_key_display_min_limit = '2023-12-30T12:00:00Z'
    time_key_display_max_limit = '2024-01-03T12:00:00Z'


with open(well_data_path, "r") as f:
    raw_data = json.load(f)

In [12]:

sample_rate = 100
sampled_data = every_nth(raw_data, sample_rate)

In [ ]:

hci_profile = read_hci_profile(sampled_data, 'hci')
hci_profile = hci_profile[hci_profile['time_key'] > time_key_display_min_limit]
hci_profile = hci_profile[hci_profile['time_key'] < time_key_display_max_limit]
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        mode="markers",
        x=hci_profile['time_key'], 
        y=hci_profile["md"], 
        marker_color = hci_profile['value'],
        marker_colorscale='Plasma',
        marker_showscale=True,
    ),
)

fig.add_vline(x=packoff_time_key, line_width=3, line_dash="dash", line_color="green", name = 'Stuckpipe detected')
fig.update_layout(title = f'{well_name} - HCI Profile',
                    title_font=dict(size=25,
                                color='blue',
                                family='Arial')
                )

fig.update_yaxes(autorange = 'reversed', title = 'bit depth (md)')
fig.update_xaxes(title = 'time key')
fig.show()
fig.write_html(f"{well_name}_hci_profile.html")


In [ ]:
points = read_hci_profile(sampled_data, 'cutting_concentration')
points = points[points['time_key'] > time_key_display_min_limit]
points = points[points['time_key'] < time_key_display_max_limit]
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        mode="markers",
        x=points['time_key'], 
        y=points["md"], 
        marker_color = points['value'],
        marker_colorscale='Plasma',
        marker_showscale=True,
    ),
)

fig.add_vline(x=packoff_time_key, line_width=3, line_dash="dash", line_color="green", name = 'Stuckpipe detected')
fig.update_layout(title = f'{well_name} Cutting Concentration Profile',
                    title_font=dict(size=25,
                                color='blue',
                                family='Arial')
                )

fig.update_yaxes(autorange = 'reversed', title = 'bit depth (md)')
fig.update_xaxes(title = 'time key')
fig.show()
fig.write_html(f"{well_name}_cc_profile.html")

In [ ]:
points = read_hci_profile(sampled_data, 'cutting_bed_height')
points = points[points['time_key'] > time_key_display_min_limit]
points = points[points['time_key'] < time_key_display_max_limit]
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        mode="markers",
        x=points['time_key'], 
        y=points["md"], 
        marker_color = points['value'],
        marker_colorscale='Plasma',
        marker_showscale=True,
    ),
)

fig.add_vline(x=packoff_time_key, line_width=3, line_dash="dash", line_color="green", name = 'Stuckpipe detected')
fig.update_layout(title = f'{well_name} - Cutting Bed Height Profile',
                    title_font=dict(size=25,
                                color='blue',
                                family='Arial')
                )

fig.update_yaxes(autorange = 'reversed', title = 'bit depth (md)')
fig.update_xaxes(title = 'time key')
fig.show()
fig.write_html(f"{well_name}_cutting_bed_height_profile.html")

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        mode="markers",
        x=points['time_key'], 
        y=points["md"], 
        marker_color = points['ecd'],
        marker_colorscale='Plasma',
        marker_showscale=True,
    ),
)

fig.add_vline(x=packoff_time_key, line_width=3, line_dash="dash", line_color="green", name = 'Stuckpipe detected')
fig.update_layout(title = 'ECD Profile',
                    title_font=dict(size=25,
                                color='blue',
                                family='Arial')
                )

fig.update_yaxes(autorange = 'reversed', title = 'bit depth (md)')
fig.update_xaxes(title = 'time key')
# fig.show()
fig.write_html(f"{well_name}_ecd_profile.html")